## has been superceeded by connector_reliefweb_disaster_summaries and connector_reliefweb_situation_reports

## This pulled data from the reliefweb api



In [1]:
import requests
import pandas as pd
import uuid
import json
import os
import uuid
import re

In [2]:
out_disaster_summaries_path = "D://projects//_external_files//reliefweb_disaster_reports//" 
#out_disaster_summaries_path = "D://projects//_external_files//surveyor//rw_disaster_preprocessed//"
#api_endpoint = 'https://api.reliefweb.int/v1/reports?appname=amcross'
disaster_endpoint = 'https://api.reliefweb.int/v1/disasters?appname=amcross'


def generate_uuid(x):
    return uuid.uuid4().hex
    

def pull_disasters():


    #set a high limit for latest in case the job doesn't run for a long time
    disaster_params = {
    'appname': 'amcross','profile': 'full','preset': 'latest','limit': 400
    #,'filter[field]': "status",'filter[value]':'ongoing'
    ,'offset':0
    }


    def send_request(params):
        # Make the API request
        response = requests.get(disaster_endpoint, params=params)
        
        # Check the status of the response
        if response.status_code == 200:
            # Parse and use the response data (in JSON format)
            data = response.json()
            return data['data']
            # for disasters if we don't return everything, can't get to the original api call
            #return data
        
        else:
            print(f"Error: {response.status_code} - {response.text}")



    def generate_filename(json):
    
        #set defaults
        rec_id = uuid.uuid4().hex
        rec_date = 'yyyy_mm_dd'
        p_country = 'country_name'
    
        try:
            rec_id = json['id']
        except:
            pass
    
        try:
            rec_date = json['fields']['date']['changed'].split("T")[0]
        except:
            pass
            
        try:
            p_country = json['fields']['primary_country']['name'].replace(" ","_").lower()
        except:
            pass
            
        return f"{rec_id}_{rec_date}_reliefweb_disaster_{p_country}.json"

   
    results = send_request(disaster_params)
    print(f"new results: {len(results)}")
    for x in results:
        fname = generate_filename(x)
        #with open(f"{fpath}{fname}", 'w') as outfile:
        with open(f"{out_disaster_summaries_path}{fname}", 'w') as outfile:
            outfile.write(json.dumps(x))


    
pull_disasters()         

new results: 400


In [36]:
fpath = "D://projects//_external_files//reliefweb_situation_reports//"
output_fpath = "D://projects//_external_files//surveyor//rw_siturep_preprocessed//"
api_endpoint = 'https://api.reliefweb.int/v1/reports?appname=amcross'
disaster_endpoint = 'https://api.reliefweb.int/v1/disasters?appname=amcross'


def pull_situation_reports():
    # first get the min and max_ids
    # with every invocation of this function, first any new
    # situation reports will be pulled, followed by
    # getting 50 older reports. In this way, the
    # historical perspective will be ever increasing
    file_ids = [f.split("_")[0] for f in os.listdir(fpath)]
    file_ids.sort()
    min_id = file_ids[0]
    max_id = file_ids[-1]
    #max_is = '4022139'

    #set a high limit for latest in case the job doesn't run for a long time
    new_params = {
    'appname': 'amcross','profile': 'full','preset': 'latest','limit': 500
    ,'query[fields][]':'format.name','query[value]':'Situation Report'
    ,'filter[field]': "id",'filter[value][from]':max_id
    }

    historical_params = {
    'appname': 'amcross','profile': 'full','preset': 'latest','limit': 5
    ,'query[fields][]':'format.name','query[value]':'Situation Report'
    ,'filter[field]': "id",'filter[value][to]':min_id
    }

    def send_request(params):
        # Make the API request
        
        response = requests.get(api_endpoint, params=params)



        if response.status_code == 200:
            # Parse and use the response data (in JSON format)
        
            data = response.json()
            #data = resp_string.json()

            return data['data']

    
        else:
            print(f"Error: {response.status_code} - {response.text}")
            #return "no data"

        





    def generate_filename(json):
    
        #set defaults
        rec_id = uuid.uuid4().hex
        rec_date = 'yyyy_mm_dd'
        p_country = 'country_name'
    
        try:
            rec_id = json['id']
        except:
            pass
    
        try:
            rec_date = json['fields']['date']['original'].split("T")[0]
        except:
            pass
            
        try:
            p_country = json['fields']['primary_country']['name'].replace(" ","_").lower()
        except:
            pass
            
        return f"{rec_id}_{rec_date}_reliefweb_{p_country}.json"

   
    results = send_request(new_params)
    print(f"new results: {len(results)-1}")
    for x in results:
        fname = generate_filename(x)
        with open(f"{fpath}{fname}", 'w') as outfile:
            outfile.write(json.dumps(x))

    results = send_request(historical_params)
    print(f"historical results: {len(results)-1}")
    for x in results:
        fname = generate_filename(x)
        with open(f"{fpath}{fname}", 'w') as outfile:
            outfile.write(json.dumps(x))
    
pull_situation_reports()           

#,'filter[field]': {"date.created": {"from":"2023-11-22T13:37:21+00:00", "to":"2023-11-24T13:37:21+00:00"}}

new results: 499
historical results: 4


## Process the situation reports

In [50]:
def extract_themes(j):
    #must pass in after levelling json up to j['fields']
    themes = j.get('theme')
    if themes is None:
        return None
        
    ts =[]
    for theme in themes:
        ts.append(theme['name'].lower())
    return '; '.join(ts)

In [51]:
#df_reliefweb_situation_report = pd.DataFrame(columns = ['record_type','source_url','glide_id','idx_para','source_level_country','source_title','source_desc',
#                                                        'source_original_text','reference_url','text','authoring_org','reported_date'])
def parse_json(j):
    #for j in json_data:
    
    #parse 
    try:
        reference_url = j['href']
            
        j = j['fields']
        #print(j)
        
        glide_id = None
        disaster = j.get('disaster')
        if disaster:
            glide_id = disaster[0]['glide']
    
        
        rec_id = j['id']
        title = j['title']
        original_text = j['body']
        link_to_doc = j['url_alias']
        file_url = j['file'][0]['url']
        primary_country_iso3 = j['primary_country']['iso3']
        primary_country = j['primary_country']['shortname']
        author_org = j['source'][0]['shortname']
        report_date = j['date']['original']
        themes = extract_themes(j)
       
        original_text_list = original_text.split("\n\n")
        idx_para = 0
        for o in original_text_list:
            row = ['situation report',reference_url,glide_id,idx_para,primary_country,title,themes,o,file_url,o,author_org,report_date]
            df_reliefweb_situation_report.loc[len(df_reliefweb_situation_report)] = row
            idx_para += 1
    except:
        pass

In [55]:
#prep receiving df
df_reliefweb_situation_report = pd.DataFrame(columns = ['record_type','source_url','glide_id','idx_para','source_level_country','source_title','source_desc',
                                                        'source_original_text','reference_url','text','authoring_org','reported_date'])

# get files on disk
files_on_disk = {}
for f in os.listdir(fpath):
    k = f.split('_')[0]
    files_on_disk[k]=f

# open file
parse_file = f"{output_fpath}reliefweb_situation_reports.xlsx"
if os.path.exists(parse_file):
    df = pd.read_excel(parse_file)
    already_done = [w.split('/')[-1] for w in list(set(df['source_url'].tolist()))]
    for f in files_on_disk:
        if f not in already_done:
            with open(f"{fpath}{files_on_disk[f]}", 'r') as file:
                json_data = json.load(file)
                parse_json(json_data)
else:
    print("file not found - will process everything")
    for f in files_on_disk:
        with open(f"{fpath}{files_on_disk[f]}", 'r') as file:
            json_data = json.load(file)
            parse_json(json_data)
            


file not found - will process everything


In [56]:
df_reliefweb_situation_report = pd.concat([df_reliefweb_situation_report, df], ignore_index=True)
#df_reliefweb_situation_report = df_reliefweb_situation_report.sort_values(by='source_url')

In [57]:
output_file = f"{output_fpath}reliefweb_situation_reports_{generate_uuid(1)}.xlsx"
#df_joined.to_excel(output_fpath)

df_reliefweb_situation_report.to_excel(output_file, index=False)

## Process the summaries

In [35]:
def parse_summary_json(j):
    #j = j['data'][0]
    

    #try:
    #reference_url = j['href']
    reference_url = ''
    file_url = ''
    themes = []
    author_org = ''
    
    rec_id = j['id']
    j = j['fields']

    status = j['status']
    
    
    #glide_id = None
    glide_id = j.get('glide')
    #print(f"glide {glide_id}")
    status = j['status']
    
   
    title = j['name']
    description = j['description']
    file_url = j['url_alias']
    #file_url = j['file'][0]['url']
    primary_country_iso3 = j['primary_country']['iso3']
    primary_country = j['primary_country']['shortname']
    #author_org = j['source'][0]['shortname']
    report_date = j['date']['changed']
    
   
    original_text_list = description.split("\n\n")
    idx_para=0
    for o in original_text_list:
        row = ['disaster summary',status,reference_url,glide_id,idx_para,primary_country,title,themes,o,file_url,o,author_org,report_date]
        df_rw_disaster_sum.loc[len(df_rw_disaster_sum)] = row
        idx_para += 1
    #except:
    #    pass

    #return df_rw_disaster_sum

#parse_summary_json(json_data)

def remove_references(text):
    #remove ([xxxx](url)) once you don't need them anymore

    pattern = r'\([^)]+\)\)\s?'  # Matches anything inside a ([xxxx](url)) pattern

    text_without_sections = re.sub(pattern, '', text)
    return text_without_sections

In [36]:
#prep receiving df
df_rw_disaster_sum = pd.DataFrame(columns = ['record_type','status','source_url','glide_id','idx_para','source_level_country','source_title','source_desc',
                                                        'source_original_text','reference_url','text','authoring_org','reported_date'])

# get files on disk
fpath = "D://projects//_external_files//reliefweb_disaster_reports//"
files_on_disk = {}
for f in os.listdir(fpath):
    k = f.split('_')[0]
    files_on_disk[k]=f

#file = files_on_disk['51520']
for f in files_on_disk:
    with open(f"{fpath}{files_on_disk[f]}", 'r') as file:
        json_data = json.load(file)
        parse_summary_json(json_data)

df_rw_disaster_sum['text'] = df_rw_disaster_sum['text'].apply(remove_references) 
df_rw_disaster_sum['authoring_org'] = 'reliefweb'

In [37]:
output_file = f"D://projects//_external_files//surveyor//rw_disaster_preprocessed//disaster_summaries_{generate_uuid(1)}.xlsx"
df_rw_disaster_sum.to_excel(output_file, index=False)
#https://reliefweb.int/disaster/ep-2016-000131-som example of a disaster that's coming down

In [38]:
, encoding='utf-8-sig'

"encoding='utf-8-sig'"

## End

In [39]:


params = {
    'appname': 'amcross'  
    ,'profile': 'full'
    ,'preset': 'latest'
    ,'limit': 5
    ,'query[fields][]':'format.name'
    ,'query[value]':'Situation Report'
    ,'filter[field]': "id"
    ,'filter[value][to]':"4017723"
    #,'filter[field]': {"id": {"to":"4017723"}}
    #,'filter[field]': {"date.created": {"from":"2023-11-22T13:37:21+00:00", "to":"2023-11-24T13:37:21+00:00"}}
    #if I remove this, seems like it returns everything
    #,'fields[include][]':['source',"format",'title',"body",'disaster.glide','primary_country']
}


api_endpoint = 'https://api.reliefweb.int/v1/reports?appname=amcross'

# Make the API request
response = requests.get(api_endpoint, params=params)

# Check the status of the response
if response.status_code == 200:
    # Parse and use the response data (in JSON format)
    data = response.json()
    situation_reports = data['data']

else:
    print(f"Error: {response.status_code} - {response.text}")

In [40]:
json_data = response.json()['data']
len(json_data)

5

In [41]:
#get min situation report id
fpath = "D://projects//_external_files//reliefweb_situation_reports//"
file_ids = [f.split("_")[0] for f in os.listdir(fpath)]
file_ids.sort()
print(file_ids)
min = file_ids[0]
max = file_ids[-1]
print(f"to: {min}")
print(f"from: {max}")



['3942951', '3942958', '3942967', '3942974', '3942979', '3942992', '3943008', '3943015', '3943042', '3943043', '3943058', '3943063', '3943077', '3943127', '3943131', '3943142', '3943155', '3943175', '3943176', '3943183', '3943200', '3943203', '3943206', '3943211', '3943229', '3943251', '3943255', '3943282', '3943301', '3943322', '3943390', '3943394', '3943396', '3943398', '3943399', '3943400', '3943401', '3943415', '3943419', '3943422', '3943447', '3943488', '3943507', '3943526', '3943538', '3943560', '3943565', '3943570', '3943572', '3943576', '3943580', '3943585', '3943593', '3943598', '3943600', '3943602', '3943604', '3943605', '3943626', '3943628', '3943647', '3943666', '3943670', '3943683', '3943730', '3943732', '3943766', '3943772', '3943831', '3943835', '3943836', '3943837', '3943838', '3943848', '3943850', '3943855', '3943858', '3943860', '3943866', '3943875', '3943885', '3943897', '3943905', '3943921', '3943929', '3943948', '3943969', '3943971', '3943973', '3943980', '3943982'

In [42]:
fpath = "D://projects//_external_files//reliefweb_situation_reports//"

def generate_filename(json):

    #set defaults
    rec_id = uuid.uuid4().hex
    rec_date = 'yyyy_mm_dd'
    p_country = 'country_name'

    try:
        rec_id = json['id']
    except:
        pass

    try:
        rec_date = json['fields']['date']['original'].split("T")[0]
    except:
        pass
        
    try:
        p_country = json['fields']['primary_country']['name'].replace(" ","_").lower()
    except:
        pass
        
    return f"{rec_id}_{rec_date}_reliefweb_{p_country}.json"


for x in json_data:
    fname = generate_filename(x)
    with open(f"{fpath}{fname}", 'w') as outfile:
        outfile.write(json.dumps(x))

In [43]:
def extract_themes(j):
    #must pass in after levelling json up to j['fields']
    themes = j.get('theme')
    if themes is None:
        return None
        
    ts =[]
    for theme in themes:
        ts.append(theme['name'].lower())
    return '; '.join(ts)

In [44]:
df_reliefweb_situation_report = pd.DataFrame(columns = ['record_type','source_url','glide_id','source_level_country','source_title','source_desc','source_original_text','reference_url','text','authoring_org','reported_date'])

for j in json_data:

    #parse 
    reference_url = j['href']
        
    j = j['fields']
    
    glide_id = None
    disaster = j.get('disaster')
    if disaster:
        glide_id = disaster[0]['glide']

    
    rec_id = j['id']
    title = j['title']
    original_text = j['body']
    link_to_doc = j['url_alias']
    file_url = j['file'][0]['url']
    primary_country_iso3 = j['primary_country']['iso3']
    primary_country = j['primary_country']['shortname']
    author_org = j['source'][0]['shortname']
    report_date = j['date']['original']
    themes = extract_themes(j)
   
    original_text_list = original_text.split("\n\n")
    for o in original_text_list:
        row = ['situation report',reference_url,glide_id,primary_country,title,themes,o,file_url,o,author_org,report_date]
        df_reliefweb_situation_report.loc[len(df_reliefweb_situation_report)] = row

In [45]:
df_reliefweb_situation_report

,record_type,source_url,glide_id,source_level_country,source_title,source_desc,source_original_text,reference_url,text,authoring_org,reported_date
0,situation report,https://api.reliefweb.int/v1/reports/4017723,None,Philippines,DSWD DROMIC Report #8 on the Mw 6.8 Earthquake...,food and nutrition; health; shelter and non-fo...,**I. Situation Overview**,https://reliefweb.int/attachments/e824c2ef-9fc...,**I. Situation Overview**,Govt. Philippines,2023-11-25T00:00:00+00:00
1,situation report,https://api.reliefweb.int/v1/reports/4017723,None,Philippines,DSWD DROMIC Report #8 on the Mw 6.8 Earthquake...,food and nutrition; health; shelter and non-fo...,"On 17 November 2023 at 4:14 PM, a 6.8-magnitud...",https://reliefweb.int/attachments/e824c2ef-9fc...,"On 17 November 2023 at 4:14 PM, a 6.8-magnitud...",Govt. Philippines,2023-11-25T00:00:00+00:00
2,situation report,https://api.reliefweb.int/v1/reports/4017723,None,Philippines,DSWD DROMIC Report #8 on the Mw 6.8 Earthquake...,food and nutrition; health; shelter and non-fo...,Date/Time: 17 Nov 2023 - 04:14:08 PM,https://reliefweb.int/attachments/e824c2ef-9fc...,Date/Time: 17 Nov 2023 - 04:14:08 PM,Govt. Philippines,2023-11-25T00:00:00+00:00
3,situation report,https://api.reliefweb.int/v1/reports/4017723,None,Philippines,DSWD DROMIC Report #8 on the Mw 6.8 Earthquake...,food and nutrition; health; shelter and non-fo...,Reported Intensities:,https://reliefweb.int/attachments/e824c2ef-9fc...,Reported Intensities:,Govt. Philippines,2023-11-25T00:00:00+00:00
4,situation report,https://api.reliefweb.int/v1/reports/4017723,None,Philippines,DSWD DROMIC Report #8 on the Mw 6.8 Earthquake...,food and nutrition; health; shelter and non-fo...,"Intensity VII - Glan, Sarangani",https://reliefweb.int/attachments/e824c2ef-9fc...,"Intensity VII - Glan, Sarangani",Govt. Philippines,2023-11-25T00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...
168,situation report,https://api.reliefweb.int/v1/reports/4017696,None,oPt,UNICEF State of Palestine Humanitarian Situati...,contributions; coordination; education; food a...,"- In the Gaza Strip, since 7 October, there ha...",https://reliefweb.int/attachments/916841f3-3fc...,"- In the Gaza Strip, since 7 October, there ha...",UNICEF,2023-11-25T00:00:00+00:00
169,situation report,https://api.reliefweb.int/v1/reports/4017696,None,oPt,UNICEF State of Palestine Humanitarian Situati...,contributions; coordination; education; food a...,- WHO reports 335 attacks on health care facil...,https://reliefweb.int/attachments/916841f3-3fc...,- WHO reports 335 attacks on health care facil...,UNICEF,2023-11-25T00:00:00+00:00
170,situation report,https://api.reliefweb.int/v1/reports/4017696,None,oPt,UNICEF State of Palestine Humanitarian Situati...,contributions; coordination; education; food a...,- UNICEF was part of the interagency mission t...,https://reliefweb.int/attachments/916841f3-3fc...,- UNICEF was part of the interagency mission t...,UNICEF,2023-11-25T00:00:00+00:00
171,situation report,https://api.reliefweb.int/v1/reports/4017696,None,oPt,UNICEF State of Palestine Humanitarian Situati...,contributions; coordination; education; food a...,- UNICEF delivered medical supplies to one hos...,https://reliefweb.int/attachments/916841f3-3fc...,- UNICEF delivered medical supplies to one hos...,UNICEF,2023-11-25T00:00:00+00:00


In [46]:
df_reliefweb_situation_report.to_csv("c://temp//100_situation_reports.csv", encoding='utf-8-sig', index=False)

In [47]:
x = uuid.uuid4().hex
x

'6d0338d89b344b5ab827c8d7a735c695'

In [48]:
https://api.reliefweb.int/v1/disasters?appname=amcross&profile=list&preset=latest&slim=1


SyntaxError: invalid syntax (1921629380.py, line 1)